In [36]:
import os
import sys
from scipy.integrate import solve_ivp
import numpy as np
from numpy.linalg import norm
from numpy import sqrt
from math import pi
import pickle
import matplotlib.pyplot as plt
from packages import data_container
from packages.data_container import Data
from packages.helper import play_trajs, rotate, sp2a, v2sp, dist, psi, beta, d_theta, d_psi, sp2v, dist, traj_speed
from packages.ode_simulator import ODESimulator
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

file = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Bai_movObst1_data.pickle'))
with open(file, 'rb') as f:
    data = pickle.load(f)

'''Models'''
fajen_approach = {'name': 'fajen_approach', 'ps': 1, 'b1': 3.25, 'k1': 7.5, 'c1': 0.4, 'c2': 0.4, 'k2': 1.4}
fajen_approach2 = {'name': 'fajen_approach2', 
                         'ps': 1.3, 'b1': 3.25, 'k1':7.5, 'c1': 0.4, 'c2': 0.4, 'b2': 4.8, 'k2': 6}
cohen_avoid = {'name': 'cohen_avoid', 'ps': 1.3, 'b1': 3.25, 'k1': 530, 'c5': 6, 'c6': 1.3, 
               'b2': 3.25, 'k2': 530, 'c7': 6, 'c8': 1.3}
cohen_avoid2 = {'name': 'cohen_avoid2', 'ps': 1.3, 'b1': 3.25, 'k1': 530, 'c5': 6, 'c6': 1.3, 
               'b2': 3.25, 'k2': 530, 'c7': 6, 'c8': 1.3}
cohen_avoid3 = {'name': 'cohen_avoid3', 'k1': 530, 'c5': 6, 'c6': 1.3, 'k2': 50, 'c7': 6, 'c8': 1.3}
cohen_avoid4 = {'name': 'cohen_avoid4', 'k1': 530, 'c5': 6, 'c6': 1.3, 'k2': 50, 'c7': 6, 'c8': 1.3}

In [69]:
set(data.info['obst_angle'])
# set(data.info['obst_speed'])

{-180.0, -157.5, -135.0, -112.5, -90.0, 0.0, 90.0, 112.5, 135.0, 157.5, 180.0}

In [6]:
'''Simulation with var0'''
%matplotlib qt
Hz = 100
xg0, yg0, xo0, yo0, x0, y0 = 0, 10, 5, 5, 0, 0
vxo0, vyo0, vx0, vy0 = -1, 0, 0, 1
s0, phi0 = v2sp([vx0, vy0])
dphi0 = dds = 0
var0 = [xg0, yg0, xo0, yo0, vxo0, vyo0, x0, y0, vx0, vy0, phi0, s0, dphi0, dds]
models = [fajen_approach, cohen_avoid]
args = {'w_goal': 0.1, 'w_obst': 0.1}
sim = ODESimulator(Hz=90, models=models, args=args, ref=[0, 1])
sim.simulate(var0, total_time=10)
sim.play()

Simulation finished in 0:00:00 t_total 9.988889


In [2]:
'''Load data to simulator'''
sim = ODESimulator(data=data, ref=[0, 1])

Loading finished


In [3]:
'''Simulate one trial'''
############
subject = 0
trial = 0
approach = {'name': 'fajen_approach', 'b1': 1.3118996599538175, 'k1': 20.05817176956081, 'c1': 2.533333934342072, 'c2': 0.07555087088494414, 'k2': 0.7413974342994933, 'ps': 1.282479471344188}
avoid = {'name': 'cohen_avoid', 'b1': 6.062816742962076, 'k1': 108.96028254157136, 'c5': 19.28855721578636, 'c6': 0.10048432127026148, 'b2': 0.005258155378802256, 'k2': 518.1796716975705, 'c7': 9.998918268499008, 'c8': 1.2379774342292225, 'ps': 1.282479471344188}
models = [approach, avoid]
############
i_trial = subject * 160 + trial - 1
%matplotlib qt
sim.reset()
sim.models = models
sim.simulate_all(trials=[i_trial], t_start='match_order', t_end='obst_out', ps='subj')
title = 'subj ' + str(sim.data.info['subj_id'][i_trial]) + \
        ' trial ' + str(sim.data.info['trial_id'][i_trial]) + \
        ' obst_angle: ' + str(sim.data.info['obst_angle'][i_trial]) + \
        ' obst_speed: ' + str(sim.data.info['obst_speed'][i_trial])
sim.play(0, actual=False, title=title)
sim.test('p_dist')

Simulated 1 trials in 0:00:00


0.46844995486384194

In [64]:
'''Simuate all trials'''
############
subject = 1
approach = {'name': 'fajen_approach', 'b1': 0.8081777207885088, 'k1': 32.80043575704669, 'c1': 8.416453555226326, 'c2': 0.03974207207189774, 'k2': 0.8136444335415792, 'ps': 1.329331601172892}
avoid = {'name': 'cohen_avoid', 'b1': 33.704853892505334, 'k1': 673.535032951836, 'c5': 19.88888904128986, 'c6': 0.19250925303961428, 'b2': 13.225629649781725, 'k2': 712.2956125001266, 'c7': 6.807300704087232, 'c8': 0.6934559494941175, 'ps': 1.329331601172892}
models = [approach, avoid]
############
%matplotlib qt
sim.reset()
sim.models = models
trials = []
for i in range(len(data.trajs)):
    if i in data.dump or data.info['subj_id'][i] != subject or data.info['obst_speed'][i] == 0:
        continue
    trials.append(i)
sim.simulate_all(trials, t_start='stimuli_onset', t_end='obst_out')
sim.test('p_dist')

Simulated 120 trials in 0:00:09


0.28449129921329525

In [72]:
'''Show one trial from batch simulations'''
######## index in simulated trials only
i = 83
########
i_trial = sim.i_trials[i]
title = 'subj ' + str(sim.data.info['subj_id'][i_trial]) + \
        ' trial ' + str(sim.data.info['trial_id'][i_trial]) + \
        ' obst_angle: ' + str(sim.data.info['obst_angle'][i_trial]) + \
        ' obst_speed: ' + str(sim.data.info['obst_speed'][i_trial])
sim.play(i, title=title, save=False)

# When beta and dpsi has the same sign it means pass in front, otherwise it means pass from behind
pass_order = sim.data.info['pass_order'][i]
pred = sim.pass_order_pred[i]
print('pass order ', pass_order, 'prediction ', pred)
print(f"err is {sim.test('p_dist', i_trial=i)}")

pass order  -1.0 prediction  1.0
err is 0.5442221570927824


In [65]:
'''true vs predicted speed by condition'''
#####################
subject = 1
#####################
%matplotlib qt
Hz = sim.Hz
fig = plt.figure()
fig.suptitle('Subject ' + str(subject))
axes = {}
obst_angle = sorted(set([abs(x) for x in data.info['obst_angle'] if x != 0]))
obst_speed = sorted(set([abs(x) for x in data.info['obst_speed'] if x != 0]))
i_plot = 1
for angle in obst_angle:
    for speed in obst_speed:
        axes[(angle, speed)] = fig.add_subplot(5, 5, i_plot)
        axes[(angle, speed)].set_xlim(0, 5)
        axes[(angle, speed)].set_ylim(0.5, 2)
        axes[(angle, speed)].set_title(str(angle) + '° ' + str(speed) + 'm/s')
#         axes[(angle, speed)].set_aspect('equal')
        i_plot += 1
for i, i_trial in enumerate(sim.i_trials):
    speed = sim.data.info['obst_speed'][i_trial]
    angle = sim.data.info['obst_angle'][i_trial]
    subj_id = sim.data.info['subj_id'][i_trial]
    if subj_id != subject or speed == 0:
        continue
    
    subj, pred = sim.p_subj[i], sim.p_pred[i]
    s_subj, s_pred = traj_speed(subj, Hz), traj_speed(pred, Hz)
    t = np.linspace(0, len(pred)-1, len(pred)) / Hz
    # Speed
    axes[(abs(angle), speed)].plot(t, s_subj, 'r')
    axes[(abs(angle), speed)].plot(t, s_pred, 'b')


In [67]:
'''true vs predicted heading by condition'''
#####################
subject = 1
#####################
%matplotlib qt
Hz = sim.Hz
fig = plt.figure()
fig.suptitle('Subject ' + str(subject))
axes = {}
obst_angle = sorted(set([abs(x) for x in data.info['obst_angle'] if x != 0]))
obst_speed = sorted(set([abs(x) for x in data.info['obst_speed'] if x != 0]))
i_plot = 1
for angle in obst_angle:
    for speed in obst_speed:
        axes[(angle, speed)] = fig.add_subplot(5, 5, i_plot)
        axes[(angle, speed)].set_xlim(0, 5)
        axes[(angle, speed)].set_ylim(0, 1.5)
        axes[(angle, speed)].set_title(str(angle) + '° ' + str(speed) + 'm/s')
#         axes[(angle, speed)].set_aspect('equal')
        i_plot += 1
for i, i_trial in enumerate(sim.i_trials):
    speed = sim.data.info['obst_speed'][i_trial]
    angle = sim.data.info['obst_angle'][i_trial]
    subj_id = sim.data.info['subj_id'][i_trial]
    if subj_id != subject or speed == 0:
        continue
    
    subj, pred = sim.p_subj[i], sim.p_pred[i]
    h_subj, h_pred = v2sp(np.gradient(subj, axis=0) * Hz)[1], v2sp(np.gradient(pred, axis=0) * Hz)[1]
    if h_subj[0] < 0:
        h_subj += pi
        h_pred += pi
    t = np.linspace(0, len(pred)-1, len(pred)) / Hz
    # Heading
    axes[(abs(angle), speed)].plot(t, h_subj, 'r')
    axes[(abs(angle), speed)].plot(t, h_pred, 'b')
    if abs(angle) == 112.5 and speed == 1.2:
        print(i, i_trial)


18 184
69 253
77 263
83 273
111 311
118 318


In [8]:
xg, yg, xo, yo, vxo, vyo, x, y, vx, vy, phi, s, dphi, ds = sim.var0_match[3]
p0, p1, v0, v1 = [x, y], [xo, yo], [vx, vy], [vxo, vyo]
beta(p0, p1, v0) * d_psi(p0, p1, v0, v1)

-8.916084053041905e-06

In [29]:
sim.test('order_accuracy')

0.65